# Comparing contextual vectors

In [ ]:
#pip install torch transformers datasets bertviz seaborn scikit-learn

In [21]:
import numpy as np
import datasets
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity

import seaborn as sns
import matplotlib.pyplot as plt
from bertviz import model_view, head_view

So far, we have been working with *static* word (or sequence) embeddings.   
Word2vec is an example of static word embeddings – it has a set vocabulary, where each word has a single embedding that is the same no matter the case.

But as we know, words can take on different meanings depending on their context. 
Contextualized embeddings aim to learn something about the semantics of the whole sequence. Practically, contextualized embeddings, like those produced by Transformer models, generate different embeddings for the same word based on its surrounding context. 

It should be possible to capture the difference between "bank" in "river bank" vs. "bank loan". 

One of the mechanisms that allows this is attention.
Let's start seeing if we can recreate the example from lecture: `"The cat sat on the mat"` with a real model.

In [3]:
sentence = "The cat sat on the mat"

In [4]:
# Load a small pretrained transformer model to extract embeddings and attention
model_name = "prajjwal1/bert-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(
    model_name, output_attentions=True, attn_implementation="eager")

config.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/116M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/116M [00:00<?, ?B/s]

In [5]:
# Tokenize
inputs = tokenizer(sentence, return_tensors='pt')
inputs

{'input_ids': tensor([[  101,  1996,  4937,  2938,  2006,  1996, 13523,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [6]:
# Run inference
with torch.no_grad():
    outputs = model(**inputs)

In [7]:
# Extract embeddings (last hidden state) and attention from the model
embeddings = outputs.last_hidden_state  
attentions = outputs.attentions  

# Decode tokens for visualization
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

In [8]:
# embeddings are a tensor shape (batch_size, sequence_length, embedding_size)
embeddings.shape

torch.Size([1, 8, 512])

In [9]:
# attentions is a tuple of tenros...
[
    type(attentions),
    len(attentions)
]


[tuple, 4]

In [10]:
# ...where each layer has (batch_size, num_heads, sequence_length, sequence_length)
attentions[0].shape

torch.Size([1, 8, 8, 8])

Let's see how that looks visualized.

This specific model has just 4 layers of attention (this is actually quite low). 
 
And 8 attention heads. This number has nothing to do with the specific sentence (which is 8 tokens long), but instead, this is decided by the model's architecture.

In [11]:
head_view(attentions, tokens)

<IPython.core.display.Javascript object>

Or you can plot each attention head separately:

In [ ]:
# Attention from the first layer, first head
L1H1 = attentions[0][0, 0]

# Plot attention weights for the CLS token (attention_scores[0] gives weights from CLS to all tokens)
plt.figure(figsize=(10, 6))
sns.heatmap(L1H1, annot=True, xticklabels=tokens, yticklabels=tokens, cmap='Blues')
plt.show()

In [ ]:
# plot all attention heads in the model
model_view(attentions, tokens)

Relevant at this point (but will be on the readings later):

- [CLS] is a special token added at the beginning of each sequence. It means “classification” and should contain some information representing the whole sequence.
- [SEP] is a special token used to separate two sequences in tasks like question answering or sentence pair classification. It also marks the end of a single sequence when only one is provided.

These special tokens are specific to the BERT architecture (Devlin et al., 2019)

Now, what do there connection mean?

1. which attention heads are covering the words that attend to themselves?
2. which heads are capturing the order of words in the sentence?
3. which heads are telling the [CLS] token what is important in the sentence?

### Comparing embeddings – Projections
Let's try to project the embeddings using dimensionality reduction to get more insight into how meaning is represented in our model.  


In [ ]:
embeddings = embeddings.squeeze(0)  # Remove the batch dimension

# Apply PCA to reduce dimensionality to 2 components
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(embeddings.cpu().numpy())

# Plot the 2D projection of BERT embeddings
plt.figure(figsize=(10, 8))
for i, token in enumerate(tokens):
    plt.scatter(embeddings_2d[i, 0], embeddings_2d[i, 1], marker='x', color='blue')
    plt.text(embeddings_2d[i, 0], embeddings_2d[i, 1], token, fontsize=12)


Notice where the embeddings landed.
What is happening here?

What characteristic do the outlier words share?  
What kind of words are close to the [CLS], central token?

### Paraphrasing – distance between sentences

The goal of the model is to embed similar sentences close to each other in the embedding space. 
A nice case for this is paraphrasing – saying (almost) the same thing using different words. 
If a model is able to detect paraphrases, this is a good indication of its performance.

So, our goal is to test whether the model understands that 
`"In 1995, the last survey, those numbers were equal."` is equivalent to `"The last time the survey was conducted, in 1995, those numbers matched."`

To do that, we will measure distance between embeddings of the two sentences.
We have to implement a distance metric first.

In [65]:
# Implement a cosine similarity in any way
def cosine_similarity(A, B)  -> float:
    """ Dot product of two vectors divided by the product of their norms (magnitudes)"""
    dot = torch.dot(A, B)
    norms = (np.linalg.norm(A)) * (np.linalg.norm(B))
    cos = dot / norms
    return cos

# test if it works
testing_results = cosine_similarity(
    A = torch.Tensor([0.5, 0.2, 0.7]),
    B = torch.Tensor([0.2, 0.1, 0.5])
    )

assert np.isclose(testing_results, 0.9716)

In [68]:
# Implement a function for extracting embeddings
def infer_embedding(sentence: str) -> torch.Tensor:
    """
    """
    inputs = tokenizer(sentence, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)

    embeddings = outputs.last_hidden_state
    embeddings = torch.squeeze(embeddings, 0)

    return embeddings

assert infer_embedding("test test test").shape == (5, 512)

In [63]:
original_sent = "In 1995, the last survey, those numbers were equal."
paraphrased_sent = "The last time the survey was conducted, in 1995, those numbers matched."

In [118]:
max_orig_sent = torch.max(emb_orig_sent, dim=0)


In [123]:
max_orig_sent[0].shape

torch.Size([512])

In [112]:
# run the little test
emb_orig_sent = infer_embedding(original_sent)
emb_para_sent = infer_embedding(paraphrased_sent)

cls_orig_sent = emb_orig_sent[0, :]
cls_para_sent = emb_para_sent[0, :]

mean_orig_sent = torch.mean(emb_orig_sent, dim=0)
mean_para_sent = torch.mean(emb_para_sent, dim=0)

max_orig_sent = torch.max(emb_orig_sent, dim=0)[0]
max_para_sent = torch.max(emb_para_sent, dim=0)[0]

print(f"Cosine similarity (CLS token): {cosine_similarity(cls_orig_sent, cls_para_sent)}")
print(f"Cosine similarity (Mean pooling): {cosine_similarity(mean_orig_sent, mean_para_sent)}")
print(f"Cosine similarity (Max): {cosine_similarity(max_orig_sent, max_para_sent)}")

Cosine similarity (CLS token): 0.8661871552467346
Cosine similarity (Mean pooling): 0.875288724899292
Cosine similarity (Max): 0.9387845396995544


Thoughts?  
Do the similarities indicate that the sentences are paraphrased?  

Let's see how representative the different measures are in a projection:

In [75]:
pca = PCA(n_components=2)
emb_orig_2d = pca.fit_transform(emb_orig_sent)
emb_para_2d = pca.transform(emb_para_sent)
cls_orig_2d = pca.transform(cls_orig_sent)
cls_para_2d = pca.transform(cls_para_sent)


plt.figure(figsize=(10, 8))
for i, token in enumerate(tokens):
    plt.scatter(emb_orig_2d[i, 0], emb_orig_2d[i, 1], marker='x', color='blue')
    plt.scatter(emb_para_2d[i, 0], emb_para_2d[i, 1], marker='x', color='blue')
    plt.scatter(cls_orig_2d[i, 0], cls_orig_2d[i, 1], marker='x', color='red')
    plt.scatter(cls_para_2d[i, 0], cls_para_2d[i, 1], marker='x', color='red')

ValueError: Expected 2D array, got 1D array instead:
array=[ 1.69728011e-01  1.34435371e-01  4.05786872e-01 -9.05322194e-01
 -3.63821894e-01 -1.31789863e-01 -5.58004797e-01 -6.39327168e-02
 -1.21476531e+00  1.26290810e+00 -1.50354218e+00 -3.57427895e-02
  5.35422087e-01 -1.18768406e+00  8.59165728e-01  3.97955626e-01
  1.09797931e+00  1.87919855e-01  1.13266420e+00  1.74052328e-01
 -9.59557474e-01  2.97035962e-01  2.08339617e-01  6.02539003e-01
  5.57833552e-01 -5.66028953e-01 -9.58719790e-01 -8.25532153e-02
  3.61107528e-01 -7.45211959e-01 -6.04736984e-01 -1.68065682e-01
 -9.81481850e-01  1.72707886e-01  5.23328364e-01  1.07361627e+00
  2.02259257e-01  2.38869831e-01 -2.47066081e-01  2.71581858e-01
 -4.08502162e-01 -4.18535948e-01 -3.79945338e-02  3.10504347e-01
 -2.35299617e-01  5.66094041e-01  8.82426679e-01 -9.62718651e-02
  1.67631999e-01  1.82238281e+00 -9.04209390e-02  1.50669765e+00
 -1.43260658e-01  5.78993380e-01 -7.25730360e-01 -3.59191775e-01
  1.99727714e-01 -3.01683605e-0

Bonus: 
- Dependency distance: does it relate to attention, check using `spacy`

### First taste of masked learning

In [76]:
# masked attention
# 1. Define the sentence
sentence = "The [MASK] sat on the mat"

# 2. Load a pretrained BERT tokenizer and model for embedding generation
model_name = "prajjwal1/bert-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, output_attentions=True)

# 3. Tokenize the sentence
inputs = tokenizer(sentence, return_tensors='pt')

# 4. Generate embeddings and attention weights using BERT
with torch.no_grad():
    outputs = model(**inputs)

# Extract hidden states (embeddings) and attention from the model
hidden_states = outputs.hidden_states  # (layer, batch, seq_len, hidden_size)
attentions = outputs.attentions  # (layer, batch, num_heads, seq_len, seq_len)

# Convert token IDs back to tokens for visualization
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

head_view(attentions, tokens)

BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


<IPython.core.display.Javascript object>

In [77]:
from transformers import pipeline

unmasker = pipeline('fill-mask', model=model_name)
unmasker("The [MASK] sat on the mat")

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


[{'score': 0.04968415945768356,
  'token': 3232,
  'token_str': 'couple',
  'sequence': 'the couple sat on the mat'},
 {'score': 0.03445810079574585,
  'token': 2158,
  'token_str': 'man',
  'sequence': 'the man sat on the mat'},
 {'score': 0.030520500615239143,
  'token': 2048,
  'token_str': 'two',
  'sequence': 'the two sat on the mat'},
 {'score': 0.02803701162338257,
  'token': 2450,
  'token_str': 'woman',
  'sequence': 'the woman sat on the mat'},
 {'score': 0.027655931189656258,
  'token': 2336,
  'token_str': 'children',
  'sequence': 'the children sat on the mat'}]

# Building a sentence similarity benchmark

- create a dataset
    - do we need all the datapoints? 
    - the data is scored on an ordinal scale, how do we turn it into a binary problem?
- pick models to compare
- embed the texts
- evaluate accuracy

In [ ]:
# load the datasets
raw_pairs = datasets.load_dataset("mteb/stsb_multi_mt", name="en", split="test")


# create a model registry
model_names = [
    ""
]

In [ ]:
raw_pairs

In [ ]:
# downsample
sample = raw_pairs.train_test_split(train_size=600, stratify_by_column="label", seed=42)

# infer
embeddings = infer_embedding()

Bonus 1:
- find edge cases (instances where the model fails)
- can edge cases be explained with dependency distance?

Bonus 2:
- compare static and contextualized embeddings on the paraphrasing task
- e.g. get representations using GloVe (Class 2, `embeddings = gensim.downloader.load("glove-wiki-gigaword-300")`)